In [ ]:
!pip install pytorch-ignite
#이벤트 방식의 프레임워크

     |████████████████████████████████| 232 kB 4.3 MB/s 


In [ ]:
import ignite #pytorch ignite

In [ ]:
print(f'ignite version : {ignite.__version__}') #version확인

ignite version : 0.4.6


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns   #example seaborn

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.contrib.handlers import ProgressBar

In [ ]:
print(torch.__version__)

1.9.0+cu102


In [ ]:
def get_data_sample():
    batch_size = 128
    transform_fn = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])
    
    trainset = datasets.FashionMNIST('./data', download=True, train=True, transform=transform_fn)
    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

    validationset = datasets.FashionMNIST('./data', download=True, train=False, transform=transform_fn)
    val_loader = DataLoader(validationset, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader



In [ ]:
train_loader, val_loader = get_data_sample()

print(train_loader.dataset)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.5, std=0.5)
           )


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
"""오타가 있어요!
class ConvolutionNeuralNet(nn.Module):

  def __init__(self):
    super(ConvolutionNeuralNet,self).__init__()

    #특징점을 잘 뽑아주는 아이
    self.feature_1 = nn.sequential(   #첫번째 특징점
        nn.Conv2d(1,32,3,padding=1),
        nn.BatchNorm2d(32),    #output과 숫자가 같아야 함 , 좀더 그룹화를 해서 해석이 가능
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=1, stride=2)   #Convolution의 특징을 잘 뽑아주는 아이 
    )
    
    self.feature_2 = nn.Sequential(   #첫번째 속에서 더 디테일한 특징점을 찾아냄
        nn.Conv2d(32,364,3),
        nn.BatchNorm2d(64),    #output과 숫자가 같아야 함 , 좀더 그룹화를 해서 해석이 가능
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=1, stride=2)   #Convolution의 특징을 잘 뽑아주는 아이
        
    )
    self.fc_1 = nn.Linear(64*6*6,600)
    self.drop = nn.Dropout2d(0.25)    #오버피팅을 피하기 위한 하나의 오퍼레이션, 요소를 지워가며 학습을 하여 일반화시킨다.
    self.fc_2 = nn.Linear(600, 120)
    self.fc_3 = nn.Linear(120, 10)    #class가 10개임

  def forward(self, input_x):
    #이미지의 특징점을 찾아주는 네트워크
    x = self.feature_1(x)
    x = self.feature_2(X)
    
    #찾아준 특징점을 가지고 이미지를 분류하는 네트워크
    x.view(-1,64*6*6)
    x = self.fc_1(x)
    x = self.drop(x)
    x = self.fc_2(x)
    x = self.fc_3(x)

    return F.log_softmax(x,dim=1)
    """


'오타가 있어요!\nclass ConvolutionNeuralNet(nn.Module):\n\n  def __init__(self):\n    super(ConvolutionNeuralNet,self).__init__()\n\n    #특징점을 잘 뽑아주는 아이\n    self.feature_1 = nn.sequential(   #첫번째 특징점\n        nn.Conv2d(1,32,3,padding=1),\n        nn.BatchNorm2d(32),    #output과 숫자가 같아야 함 , 좀더 그룹화를 해서 해석이 가능\n        nn.ReLU(),\n        nn.MaxPool2d(kernel_size=1, stride=2)   #Convolution의 특징을 잘 뽑아주는 아이 \n    )\n    \n    self.feature_2 = nn.Sequential(   #첫번째 속에서 더 디테일한 특징점을 찾아냄\n        nn.Conv2d(32,364,3),\n        nn.BatchNorm2d(64),    #output과 숫자가 같아야 함 , 좀더 그룹화를 해서 해석이 가능\n        nn.ReLU(),\n        nn.MaxPool2d(kernel_size=1, stride=2)   #Convolution의 특징을 잘 뽑아주는 아이\n        \n    )\n    self.fc_1 = nn.Linear(64*6*6,600)\n    self.drop = nn.Dropout2d(0.25)    #오버피팅을 피하기 위한 하나의 오퍼레이션, 요소를 지워가며 학습을 하여 일반화시킨다.\n    self.fc_2 = nn.Linear(600, 120)\n    self.fc_3 = nn.Linear(120, 10)    #class가 10개임\n\n  def forward(self, input_x):\n    #이미지의 특징점을 찾아주는 네트워크\n    x = self.feature_1(x)\n   

In [ ]:
class ConvolutionNeuralNet(nn.Module):
    
    def __init__(self):
        super(ConvolutionNeuralNet, self).__init__()
        self.feature_1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.feature_2 = nn.Sequential(
            nn.Conv2d(32, 64, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc_1 = nn.Linear(64 * 6 * 6, 600)
        self.drop = nn.Dropout2d(0.25)
        self.fc_2 = nn.Linear(600, 120)
        self.fc_3 = nn.Linear(120, 10)

    def forward(self, input_x):
        """
            이미지의 특징점을 찾아주는 네트워크
        """
        x = self.feature_1(input_x)
        x = self.feature_2(x)
        """
            찾아준 특징점을 가지고 이미지를 분류하는 네트워크
        """
        x = x.view(-1, 64 * 6 * 6)
        x = self.fc_1(x)
        x = self.drop(x)
        x = self.fc_2(x)
        x = self.fc_3(x)

        return F.log_softmax(x, dim=1)

In [ ]:
train_loader,val_loader = get_data_sample()

In [ ]:
model=ConvolutionNeuralNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optim = optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = nn.NLLLoss()


In [ ]:
"""
    학습을할 operation 평가를할 operation
"""
metrics = {
    'accuracy': Accuracy(),
    'nll': Loss(loss_fn),
    'cm': ConfusionMatrix(num_classes=10)
}
trainer = create_supervised_trainer(model, optim, loss_fn, device=device)
"""
    train_evaluator: 현재 학습하는데이터로 평가를 했을 경우
    val_evaluator:  학데이터로 평가를 했을 경우
"""
train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)

training_history = {'accuracy': [], 'loss': []}
validation_history = {'accuracy': [], 'loss': []}

last_epoch = []

In [ ]:
RunningAverage(output_transform=lambda x: x).attach(trainer,'loss')

In [ ]:
def score_function(engine):
  val_loss = engine.state.metrics['nll']    #음수로 나온다.
  return -val_loss

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED) #@는 레지스터
def log_training_results(trainer):
  train_evaluator.run(train_loader)

  matrics = train_evaluator.state.metrics
  accuracy = metrics['accuracy']*100
  loss = metrics['nll']
  last_epoch.append(0)
  #log visualzation(lossfunction value, accuracy value)
  training_history['accuracy'].append(accuracy)
  training_history['loss'].append(loss)
  print(f"Training Results - Epoch {trainer.state.epoch} Avg accuracy : {accuracy}, Avg loss : {loss: .4f}")



In [ ]:
  def log_validation_results(trainer):
    val_evaluator.run(val_loader)

    metrics = val_evaluator.state.metrics
    accuracy = metrics['accuracy'] * 100
    loss = metrics['nll']

    validation_history['accuracy'].append(accuracy)
    validation_history['loss'].append(loss)
    print(f"Validation Results - Epoch: {val_evaluator.state.epoch} Avg accuracy: {accuracy}, Avg loss: {loss:.4f}")


In [ ]:
@trainer.on(Events.COMPLETED)
def log_confusion_matrix(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    cm = metrics['cm']
    cm = cm.numpy()
    cm = cm.astype(int)
    classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot']
    fig, ax = plt.subplot(figsize=(10, 10))
    ax = plt.subplots()
    sns.heatmap(cm, annot=True, ax=ax, fmt="d")
    """
        Ture Label, Prediction Label
    """
    ax.set_xlabel('prediction labels')
    ax.set_ylabel('true labels')
    ax.set_title('confusion matirx')
    ax.xaxis.set_ticklabels(classes, rotation=90)
    ax.yaxis.set_ticklabels(classes, rotation=0)

In [ ]:
handler = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, handler)
trainer.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)
checkpointer = ModelCheckpoint('./saved_models', 'fashionMNIST', n_saved=2, create_dir=True, save_as_state_dict=True, require_empty=False)

In [ ]:
ProgressBar(persist=True).attach(trainer, output_transform=lambda x: {"batch loss": x})
trainer.run(train_loader, max_epochs=5)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1/469]   0%|           [00:00<?]

Engine run is terminating due to exception: unsupported format string passed to Loss.__format__


TypeError: ignored